# (Research to) Build the W-transit Reference Table
- Can ignore this page completely for final model and final analysis... only keep as reference for code examples
---

## FINDINGS
- York Mills has standard N, W, S, E stops at York Mills & Yonge intersection which kick off / end most trips heading to and from York Mills Station...
- Only exception is the Subway lines which takes off from the Northbound / Southbound Platform stops
- Finch station however has one unique bus stop number for each route that arrives and departs from that station
- Can safely delete all routes with a route long name that includes 'night' or 'express'
- In particular, should defintely delete night service
- Express might make sense to keep but can be deleted as well
- Investigated York Mills route in detail...
- All stops are in sequence and follow each other, hence safe to take the average time between any two stops; and cumulative time for any sequence of stops
---

In [1]:
import pandas as pd

In [2]:
f = pd.read_pickle('data/model/f.pickle')
stops = pd.read_feather('data/model/stops.ftr')
seq = pd.read_feather('data/model/shape_sequence_durations.ftr')

In [3]:
shapes = pd.read_feather('data/store/shapes.ftr')

In [4]:
schedule = pd.read_feather('data/model/schedule.ftr')

In [5]:
schedule

,trip_id,route_short_name,route_long_name,shape_id,trip_headsign,stop_sequence,stop_time,stop_time_delta,stop_id,stop_code,stop_name,stop_lat,stop_lon
0,42990004,10,VAN HORNE,886387,EAST - 10 VAN HORNE towards VICTORIA PARK,1,7:00:29,0 days 07:00:29,14155,14633,Don Mills Station,43.776222,-79.347048
1,42990004,10,VAN HORNE,886387,EAST - 10 VAN HORNE towards VICTORIA PARK,2,7:01:19,0 days 07:01:19,3807,1949,Don Mills Rd at Leith Hill Rd North Side,43.777534,-79.347811
2,42990004,10,VAN HORNE,886387,EAST - 10 VAN HORNE towards VICTORIA PARK,3,7:02:12,0 days 07:02:12,6904,1929,Don Mills Rd at Fairview Mall Dr North Side,43.779530,-79.348701
3,42990004,10,VAN HORNE,886387,EAST - 10 VAN HORNE towards VICTORIA PARK,4,7:03:35,0 days 07:03:35,1163,1938,Don Mills Rd at Godstone Rd,43.782682,-79.348922
4,42990004,10,VAN HORNE,886387,EAST - 10 VAN HORNE towards VICTORIA PARK,5,7:04:48,0 days 07:04:48,7723,1919,Don Mills Rd at Deerford Rd,43.785281,-79.350570
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1304777,43128797,4,LINE 4 (SHEPPARD),890357,LINE 4 (SHEPPARD) towards SHEPPARD-YONGE STATION,1,5:31:00,0 days 05:31:00,14535,14109,Don Mills Station - Westbound Platform,43.775248,-79.346189
1304778,43128797,4,LINE 4 (SHEPPARD),890357,LINE 4 (SHEPPARD) towards SHEPPARD-YONGE STATION,2,5:33:05,0 days 05:33:05,14536,13847,Leslie Station - Westbound Platform,43.771248,-79.366790
1304779,43128797,4,LINE 4 (SHEPPARD),890357,LINE 4 (SHEPPARD) towards SHEPPARD-YONGE STATION,3,5:34:50,0 days 05:34:50,14537,13846,Bessarion Station - Westbound Platform,43.769296,-79.376345
1304780,43128797,4,LINE 4 (SHEPPARD),890357,LINE 4 (SHEPPARD) towards SHEPPARD-YONGE STATION,4,5:36:43,0 days 05:36:43,14538,13843,Bayview Station - Westbound Platform,43.766951,-79.386730


In [6]:
f

1.4267500000002541e-05

In [7]:
stops = stops.set_index('stop_id')

In [8]:
stops.head(1)

,stop_code,stop_name,stop_lat,stop_lon
stop_id,,,,
262,662,Danforth Rd at Kennedy Rd,43.714379,-79.260939


In [9]:
seq.head(1)

,shape_id,stop_sequence,next_stop_duration,next_stop_seconds
0,886387,1,0 days 00:00:50,50.0


In [10]:
seq

,shape_id,stop_sequence,next_stop_duration,next_stop_seconds
0,886387,1,0 days 00:00:50,50.0
1,886387,2,0 days 00:00:53,53.0
2,886387,3,0 days 00:01:23,83.0
3,886387,4,0 days 00:01:13,73.0
4,886387,5,0 days 00:01:19,79.0
...,...,...,...,...
34997,890357,1,0 days 00:02:05,125.0
34998,890357,2,0 days 00:01:45,105.0
34999,890357,3,0 days 00:01:53,113.0
35000,890357,4,0 days 00:02:41,161.0


In [11]:
seq = seq.merge(schedule, how = 'left', on = ['shape_id', 'stop_sequence']).drop(columns = ['trip_id', 'stop_time', 'stop_time_delta'])

In [12]:
seq = seq.drop_duplicates()

In [13]:
seq

,shape_id,stop_sequence,next_stop_duration,next_stop_seconds,route_short_name,route_long_name,trip_headsign,stop_id,stop_code,stop_name,stop_lat,stop_lon
0,886387,1,0 days 00:00:50,50.0,10,VAN HORNE,EAST - 10 VAN HORNE towards VICTORIA PARK,14155,14633,Don Mills Station,43.776222,-79.347048
32,886387,2,0 days 00:00:53,53.0,10,VAN HORNE,EAST - 10 VAN HORNE towards VICTORIA PARK,3807,1949,Don Mills Rd at Leith Hill Rd North Side,43.777534,-79.347811
64,886387,3,0 days 00:01:23,83.0,10,VAN HORNE,EAST - 10 VAN HORNE towards VICTORIA PARK,6904,1929,Don Mills Rd at Fairview Mall Dr North Side,43.779530,-79.348701
96,886387,4,0 days 00:01:13,73.0,10,VAN HORNE,EAST - 10 VAN HORNE towards VICTORIA PARK,1163,1938,Don Mills Rd at Godstone Rd,43.782682,-79.348922
128,886387,5,0 days 00:01:19,79.0,10,VAN HORNE,EAST - 10 VAN HORNE towards VICTORIA PARK,7723,1919,Don Mills Rd at Deerford Rd,43.785281,-79.350570
...,...,...,...,...,...,...,...,...,...,...,...,...
1303637,890357,1,0 days 00:02:05,125.0,4,LINE 4 (SHEPPARD),LINE 4 (SHEPPARD) towards SHEPPARD-YONGE STATION,14535,14109,Don Mills Station - Westbound Platform,43.775248,-79.346189
1303866,890357,2,0 days 00:01:45,105.0,4,LINE 4 (SHEPPARD),LINE 4 (SHEPPARD) towards SHEPPARD-YONGE STATION,14536,13847,Leslie Station - Westbound Platform,43.771248,-79.366790
1304095,890357,3,0 days 00:01:53,113.0,4,LINE 4 (SHEPPARD),LINE 4 (SHEPPARD) towards SHEPPARD-YONGE STATION,14537,13846,Bessarion Station - Westbound Platform,43.769296,-79.376345
1304324,890357,4,0 days 00:02:41,161.0,4,LINE 4 (SHEPPARD),LINE 4 (SHEPPARD) towards SHEPPARD-YONGE STATION,14538,13843,Bayview Station - Westbound Platform,43.766951,-79.386730


In [14]:
seq['next_stop_id'] = seq.stop_id.shift(-1)

In [15]:
seq

,shape_id,stop_sequence,next_stop_duration,next_stop_seconds,route_short_name,route_long_name,trip_headsign,stop_id,stop_code,stop_name,stop_lat,stop_lon,next_stop_id
0,886387,1,0 days 00:00:50,50.0,10,VAN HORNE,EAST - 10 VAN HORNE towards VICTORIA PARK,14155,14633,Don Mills Station,43.776222,-79.347048,3807.0
32,886387,2,0 days 00:00:53,53.0,10,VAN HORNE,EAST - 10 VAN HORNE towards VICTORIA PARK,3807,1949,Don Mills Rd at Leith Hill Rd North Side,43.777534,-79.347811,6904.0
64,886387,3,0 days 00:01:23,83.0,10,VAN HORNE,EAST - 10 VAN HORNE towards VICTORIA PARK,6904,1929,Don Mills Rd at Fairview Mall Dr North Side,43.779530,-79.348701,1163.0
96,886387,4,0 days 00:01:13,73.0,10,VAN HORNE,EAST - 10 VAN HORNE towards VICTORIA PARK,1163,1938,Don Mills Rd at Godstone Rd,43.782682,-79.348922,7723.0
128,886387,5,0 days 00:01:19,79.0,10,VAN HORNE,EAST - 10 VAN HORNE towards VICTORIA PARK,7723,1919,Don Mills Rd at Deerford Rd,43.785281,-79.350570,2498.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1303637,890357,1,0 days 00:02:05,125.0,4,LINE 4 (SHEPPARD),LINE 4 (SHEPPARD) towards SHEPPARD-YONGE STATION,14535,14109,Don Mills Station - Westbound Platform,43.775248,-79.346189,14536.0
1303866,890357,2,0 days 00:01:45,105.0,4,LINE 4 (SHEPPARD),LINE 4 (SHEPPARD) towards SHEPPARD-YONGE STATION,14536,13847,Leslie Station - Westbound Platform,43.771248,-79.366790,14537.0
1304095,890357,3,0 days 00:01:53,113.0,4,LINE 4 (SHEPPARD),LINE 4 (SHEPPARD) towards SHEPPARD-YONGE STATION,14537,13846,Bessarion Station - Westbound Platform,43.769296,-79.376345,14538.0
1304324,890357,4,0 days 00:02:41,161.0,4,LINE 4 (SHEPPARD),LINE 4 (SHEPPARD) towards SHEPPARD-YONGE STATION,14538,13843,Bayview Station - Westbound Platform,43.766951,-79.386730,14539.0


In [16]:
seq[seq.stop_id == 917]

,shape_id,stop_sequence,next_stop_duration,next_stop_seconds,route_short_name,route_long_name,trip_headsign,stop_id,stop_code,stop_name,stop_lat,stop_lon,next_stop_id
533580,887271,53,0 days 00:00:24.500000,24.500000,395,YORK MILLS NIGHT BUS,WEST - 395 YORK MILLS towards YORK MILLS STATION,917,9083,York Mills Rd at Sandover Dr (1222 York Mills),43.759813,-79.331751,5191.0
1187485,888274,48,0 days 00:00:31.500000,31.500000,95,YORK MILLS,WEST - 95 YORK MILLS towards YORK MILLS STATION,917,9083,York Mills Rd at Sandover Dr (1222 York Mills),43.759813,-79.331751,5191.0
1191050,888280,17,0 days 00:00:35.500000,35.500000,95,YORK MILLS,WEST - 95C YORK MILLS towards YORK MILLS STATION,917,9083,York Mills Rd at Sandover Dr (1222 York Mills),43.759813,-79.331751,5191.0
1193169,888281,39,0 days 00:00:34.666666666,34.666667,95,YORK MILLS,WEST - 95 YORK MILLS towards YORK MILLS STATION,917,9083,York Mills Rd at Sandover Dr (1222 York Mills),43.759813,-79.331751,5191.0
1255551,888390,21,0 days 00:04:02.200000,242.200000,995,YORK MILLS EXPRESS,WEST - 995 YORK MILLS EXPRESS towards YORK MIL...,917,9083,York Mills Rd at Sandover Dr (1222 York Mills),43.759813,-79.331751,9834.0


In [17]:
seq[(seq.stop_id == 917) & (seq.next_stop_id == 5191)].next_stop_seconds.mean()

31.5416666665

In [24]:
ym_shapes = seq[seq.shape_id.isin([887271, 888274, 888280, 888281, 888390])]

In [25]:
ym_shapes

,shape_id,stop_sequence,next_stop_duration,next_stop_seconds,route_short_name,route_long_name,trip_headsign,stop_id,stop_code,stop_name,stop_lat,stop_lon,next_stop_id
533268,887271,1,0 days 00:00:09,9.0,395,YORK MILLS NIGHT BUS,WEST - 395 YORK MILLS towards YORK MILLS STATION,4343,9343,Meadowvale Loop at Sheppard Ave East,43.809148,-79.170722,574.0
533274,887271,2,0 days 00:00:28,28.0,395,YORK MILLS NIGHT BUS,WEST - 395 YORK MILLS towards YORK MILLS STATION,574,6575,Meadowvale Rd at Sheppard Ave East,43.808582,-79.171055,5746.0
533280,887271,3,0 days 00:00:39,39.0,395,YORK MILLS NIGHT BUS,WEST - 395 YORK MILLS towards YORK MILLS STATION,5746,6579,Meadowvale Rd at Rotary Dr,43.806818,-79.170155,4306.0
533286,887271,4,0 days 00:02:10,130.0,395,YORK MILLS NIGHT BUS,WEST - 395 YORK MILLS towards YORK MILLS STATION,4306,6578,Meadowvale Rd at Dean Park Rd,43.804327,-79.168982,5782.0
533292,887271,5,0 days 00:00:37,37.0,395,YORK MILLS NIGHT BUS,WEST - 395 YORK MILLS towards YORK MILLS STATION,5782,6568,Meadowvale Rd at Euclid Ave,43.796107,-79.165195,6652.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1255609,888390,22,0 days 00:03:51.400000,231.4,995,YORK MILLS EXPRESS,WEST - 995 YORK MILLS EXPRESS towards YORK MIL...,9834,9071,York Mills Rd at Don Mills Rd,43.756053,-79.346600,944.0
1255667,888390,23,0 days 00:05:08.800000,308.8,995,YORK MILLS EXPRESS,WEST - 995 YORK MILLS EXPRESS towards YORK MIL...,944,9077,York Mills Rd at Leslie St West Side,43.753001,-79.360766,1356.0
1255725,888390,24,0 days 00:03:46,226.0,995,YORK MILLS EXPRESS,WEST - 995 YORK MILLS EXPRESS towards YORK MIL...,1356,9069,York Mills Rd at Bayview Ave West Side,43.748013,-79.384820,919.0
1255783,888390,25,0 days 00:00:35,35.0,995,YORK MILLS EXPRESS,WEST - 995 YORK MILLS EXPRESS towards YORK MIL...,919,9906,York Mills Rd at Yonge St,43.744417,-79.405968,24358.0


In [26]:
ym_shapes.stop_id.value_counts()

4604     5
944      5
7360     5
5470     5
353      5
        ..
6652     1
5782     1
4306     1
5746     1
24358    1
Name: stop_id, Length: 81, dtype: int64

In [29]:
seq.stop_id.value_counts().sort_values()

14539     1
2028      1
6407      1
7128      1
3074      1
         ..
10109    19
5935     19
15211    20
3169     23
9276     23
Name: stop_id, Length: 9419, dtype: int64

In [30]:
seq[seq.stop_id == 9276]

,shape_id,stop_sequence,next_stop_duration,next_stop_seconds,route_short_name,route_long_name,trip_headsign,stop_id,stop_code,stop_name,stop_lat,stop_lon,next_stop_id
162905,886603,2,0 days 00:00:33,33.000000,129,MCCOWAN NORTH,NORTH - 129B MCCOWAN NORTH towards STEELES,9276,9614,Triton Rd at McCowan Rd,43.775294,-79.252967,2541.0
165997,886604,2,0 days 00:00:33,33.000000,129,MCCOWAN NORTH,NORTH - 129A MCCOWAN NORTH towards MAJOR MACKE...,9276,9614,Triton Rd at McCowan Rd,43.775294,-79.252967,2541.0
172026,886611,2,0 days 00:00:26.500000,26.500000,130,MIDDLEFIELD,NORTH - 130B MIDDLEFIELD towards STEELES via M...,9276,9614,Triton Rd at McCowan Rd,43.775294,-79.252967,2541.0
172433,886612,2,0 days 00:00:22,22.000000,130,MIDDLEFIELD,NORTH - 130A MIDDLEFIELD towards STEELES,9276,9614,Triton Rd at McCowan Rd,43.775294,-79.252967,2541.0
176966,886621,2,0 days 00:00:27,27.000000,131,NUGGET,EAST - 131 NUGGET towards OLD FINCH/MORNINGVIEW,9276,9614,Triton Rd at McCowan Rd,43.775294,-79.252967,2541.0
178918,886625,2,0 days 00:00:27,27.000000,131,NUGGET,EAST - 131 NUGGET towards OLD FINCH/MORNINGVIEW,9276,9614,Triton Rd at McCowan Rd,43.775294,-79.252967,2541.0
182547,886630,2,0 days 00:00:23.500000,23.500000,132,MILNER,EAST - 132 MILNER towards MCLEVIN & HUPFIELD,9276,9614,Triton Rd at McCowan Rd,43.775294,-79.252967,2541.0
188636,886635,2,0 days 00:02:11,131.000000,133,NEILSON,NORTH - 133 NEILSON towards MORNINGSIDE HEIGHTS,9276,9614,Triton Rd at McCowan Rd,43.775294,-79.252967,351.0
189970,886636,2,0 days 00:01:22,82.000000,133,NEILSON,NORTH - 133 NEILSON towards MORNINGSIDE HEIGHT...,9276,9614,Triton Rd at McCowan Rd,43.775294,-79.252967,351.0
190950,886637,2,0 days 00:01:21.500000,81.500000,133,NEILSON,NORTH - 133 NEILSON towards MORNINGSIDE HEIGHT...,9276,9614,Triton Rd at McCowan Rd,43.775294,-79.252967,351.0


In [40]:
stops[stops.stop_name.str.contains('York Mills Station')]

,stop_code,stop_name,stop_lat,stop_lon
stop_id,,,,
2036,9065,Yonge St at York Mills Rd North Side (York Mil...,43.744863,-79.406660
9756,9066,Yonge St at Wilson Ave (York Mills Station),43.744290,-79.406957
14407,13792,York Mills Station - Southbound Platform,43.743248,-79.405991
14464,13791,York Mills Station - Northbound Platform,43.744448,-79.406191


In [37]:
seq[seq.stop_id == 14407]

,shape_id,stop_sequence,next_stop_duration,next_stop_seconds,route_short_name,route_long_name,trip_headsign,stop_id,stop_code,stop_name,stop_lat,stop_lon,next_stop_id
1258779,889522,4,0 days 00:03:11,191.0,1,LINE 1 (YONGE-UNIVERSITY),LINE 1 (YONGE-UNIVERSITY) towards VAUGHAN METR...,14407,13792,York Mills Station - Southbound Platform,43.743248,-79.405991,14408.0
1260258,889575,4,0 days 00:03:11,191.0,1,LINE 1 (YONGE-UNIVERSITY),LINE 1 (YONGE-UNIVERSITY) towards SHEPPARD WES...,14407,13792,York Mills Station - Southbound Platform,43.743248,-79.405991,14408.0
1260273,889597,4,0 days 00:03:11,191.0,1,LINE 1 (YONGE-UNIVERSITY),LINE 1 (YONGE-UNIVERSITY) towards VAUGHAN METR...,14407,13792,York Mills Station - Southbound Platform,43.743248,-79.405991,14408.0
1260311,889598,4,0 days 00:03:13,193.0,1,LINE 1 (YONGE-UNIVERSITY),LINE 1 (YONGE-UNIVERSITY) towards VAUGHAN METR...,14407,13792,York Mills Station - Southbound Platform,43.743248,-79.405991,14408.0
1261126,889599,4,0 days 00:03:11,191.0,1,LINE 1 (YONGE-UNIVERSITY),LINE 1 (YONGE-UNIVERSITY) towards VAUGHAN METR...,14407,13792,York Mills Station - Southbound Platform,43.743248,-79.405991,14408.0
1270268,889620,4,0 days 00:03:15,195.0,1,LINE 1 (YONGE-UNIVERSITY),LINE 1 (YONGE-UNIVERSITY) towards VAUGHAN METR...,14407,13792,York Mills Station - Southbound Platform,43.743248,-79.405991,14408.0


In [38]:
stops

,stop_code,stop_name,stop_lat,stop_lon
stop_id,,,,
262,662,Danforth Rd at Kennedy Rd,43.714379,-79.260939
263,929,Davenport Rd at Bedford Rd,43.674448,-79.399659
264,940,Davenport Rd at Dupont St,43.675511,-79.401938
265,1871,Davisville Ave at Cleveland St,43.702088,-79.378112
266,11700,Disco Rd at Attwell Dr,43.701362,-79.594843
...,...,...,...,...
24418,16031,Gerrard St East at River St,43.663948,-79.358666
24419,16032,Broadview Ave at Gerrard St East North Side,43.665766,-79.352651
24420,16033,Jack Layton Way at St Matthews Rd,43.665695,-79.354843


In [41]:
stops['distance'] = (abs(stops.stop_lat - 43.744448)*2 + abs(stops.stop_lon - -79.406191)*2)*(1/2)

In [46]:
seq[seq.stop_id == 919]

,shape_id,stop_sequence,next_stop_duration,next_stop_seconds,route_short_name,route_long_name,trip_headsign,stop_id,stop_code,stop_name,stop_lat,stop_lon,next_stop_id
99467,886516,29,0 days 00:00:38.500000,38.500000,115,SILVER HILLS,WEST - 115 SILVER HILLS towards YORK MILLS STA...,919,9906,York Mills Rd at Yonge St,43.744417,-79.405968,24356.0
138738,886573,39,0 days 00:00:45.500000,45.500000,122,GRAYDON HALL,WEST - 122 GRAYDON HALL towards YORK MILLS STA...,919,9906,York Mills Rd at Yonge St,43.744417,-79.405968,24356.0
140158,886574,33,0 days 00:00:38.500000,38.500000,122,GRAYDON HALL,WEST - 122 GRAYDON HALL towards YORK MILLS STA...,919,9906,York Mills Rd at Yonge St,43.744417,-79.405968,24356.0
140271,886575,28,0 days 00:00:35,35.000000,122,GRAYDON HALL,WEST - 122 GRAYDON HALL towards YORK MILLS STA...,919,9906,York Mills Rd at Yonge St,43.744417,-79.405968,24356.0
533706,887271,74,0 days 00:00:29,29.000000,395,YORK MILLS NIGHT BUS,WEST - 395 YORK MILLS towards YORK MILLS STATION,919,9906,York Mills Rd at Yonge St,43.744417,-79.405968,24357.0
1002964,888030,22,0 days 00:00:37,37.000000,78,ST. ANDREWS,WEST - 78 ST ANDREWS towards YORK MILLS STATION,919,9906,York Mills Rd at Yonge St,43.744417,-79.405968,24355.0
1189732,888274,69,0 days 00:00:29.333333333,29.333333,95,YORK MILLS,WEST - 95 YORK MILLS towards YORK MILLS STATION,919,9906,York Mills Rd at Yonge St,43.744417,-79.405968,24357.0
1192499,888280,38,0 days 00:00:37,37.000000,95,YORK MILLS,WEST - 95C YORK MILLS towards YORK MILLS STATION,919,9906,York Mills Rd at Yonge St,43.744417,-79.405968,24357.0
1193463,888281,60,0 days 00:00:33,33.000000,95,YORK MILLS,WEST - 95 YORK MILLS towards YORK MILLS STATION,919,9906,York Mills Rd at Yonge St,43.744417,-79.405968,24357.0
1255783,888390,25,0 days 00:00:35,35.000000,995,YORK MILLS EXPRESS,WEST - 995 YORK MILLS EXPRESS towards YORK MIL...,919,9906,York Mills Rd at Yonge St,43.744417,-79.405968,24358.0


In [45]:
stops.sort_values(by = 'distance').head(50)

,stop_code,stop_name,stop_lat,stop_lon,distance
stop_id,,,,,
14464,13791,York Mills Station - Northbound Platform,43.744448,-79.406191,0.000000
919,9906,York Mills Rd at Yonge St,43.744417,-79.405968,0.000254
918,9907,York Mills Rd at Yonge St East Side,43.744162,-79.405826,0.000651
2036,9065,Yonge St at York Mills Rd North Side (York Mil...,43.744863,-79.406660,0.000884
910,11394,Yonge St at York Mills Rd,43.743783,-79.406435,0.000909
9756,9066,Yonge St at Wilson Ave (York Mills Station),43.744290,-79.406957,0.000924
24358,15975,York Mills Temporary Bus Loop,43.744536,-79.407150,0.001047
6062,11334,York Mills Rd at Old York Mills Rd,43.744461,-79.404964,0.001240
24357,15974,York Mills Temporary Bus Loop,43.744303,-79.407324,0.001278


In [52]:
stops[stops.stop_code == 14687]

,stop_code,stop_name,stop_lat,stop_lon,distance
stop_id,,,,,
14207,14687,Finch Station,43.780619,-79.414857,0.044837


In [55]:
finch_stn = stops[stops.stop_name == 'Finch Station'].index.to_list()

In [58]:
finch_stn

[14203, 14204, 14205, 14206, 14207, 14208, 14209, 14210, 14211, 14212, 14684]

In [61]:
seq[seq.stop_id.isin([14205])]

,shape_id,stop_sequence,next_stop_duration,next_stop_seconds,route_short_name,route_long_name,trip_headsign,stop_id,stop_code,stop_name,stop_lat,stop_lon,next_stop_id
1156856,888241,14,0 days 00:09:51,591.000000,939,FINCH EXPRESS,WEST - 939B FINCH EXPRESS towards FINCH WEST S...,14205,14685,Finch Station,43.780467,-79.414636,6073.0
1160143,888243,15,0 days 00:08:21.666666666,501.666667,939,FINCH EXPRESS,WEST - 939B FINCH EXPRESS towards FINCH WEST S...,14205,14685,Finch Station,43.780467,-79.414636,6073.0


In [70]:
seq[seq.route_long_name.str.lower().str.contains('express')]

,shape_id,stop_sequence,next_stop_duration,next_stop_seconds,route_short_name,route_long_name,trip_headsign,stop_id,stop_code,stop_name,stop_lat,stop_lon,next_stop_id
1110997,888160,1,0 days 00:12:54.285714285,774.285714,900,AIRPORT EXPRESS,SOUTH - 900 AIRPORT EXPRESS towards KIPLING ST...,3169,14278,Pearson Airport Terminal 3 (Arrivals Level Col...,43.686811,-79.620932,5407.0
1111133,888160,2,0 days 00:01:58,118.000000,900,AIRPORT EXPRESS,SOUTH - 900 AIRPORT EXPRESS towards KIPLING ST...,5407,2148,Dundas St West at The East Mall Cres,43.630370,-79.549175,14238.0
1111269,888160,3,NaT,NaN,900,AIRPORT EXPRESS,SOUTH - 900 AIRPORT EXPRESS towards KIPLING ST...,14238,14719,Kipling Station,43.637582,-79.535677,3686.0
1111405,888163,1,0 days 00:01:26,86.000000,900,AIRPORT EXPRESS,NORTH - 900 AIRPORT EXPRESS towards PEARSON AI...,3686,2138,Dundas St West at Acorn Ave,43.637518,-79.539041,6721.0
1111407,888163,2,0 days 00:13:03,783.000000,900,AIRPORT EXPRESS,NORTH - 900 AIRPORT EXPRESS towards PEARSON AI...,6721,2149,Dundas St West at The East Mall Cres,43.630726,-79.549041,14957.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1258247,888392,15,0 days 00:03:11.500000,191.500000,996,WILSON EXPRESS,WEST - 996 WILSON EXPRESS towards HUMBER COLLEGE,4074,7569,Brookmere Rd at Kipling Ave,43.728245,-79.574410,563.0
1258317,888392,16,0 days 00:02:17.333333333,137.333333,996,WILSON EXPRESS,WEST - 996 WILSON EXPRESS towards HUMBER COLLEGE,563,7957,Westhumber Blvd at Martin Grove Rd North Side,43.726355,-79.586705,9527.0
1258387,888392,17,0 days 00:03:06.666666666,186.666667,996,WILSON EXPRESS,WEST - 996 WILSON EXPRESS towards HUMBER COLLEGE,9527,7875,John Garland Blvd at Martin Grove Rd West Side,43.732009,-79.589827,5685.0
1258457,888392,18,0 days 00:02:19,139.000000,996,WILSON EXPRESS,WEST - 996 WILSON EXPRESS towards HUMBER COLLEGE,5685,7867,Humber College Blvd at Westmore Dr (Etobicoke ...,43.731137,-79.598867,15248.0


In [75]:
seq[seq.next_stop_id == 3169]

,shape_id,stop_sequence,next_stop_duration,next_stop_seconds,route_short_name,route_long_name,trip_headsign,stop_id,stop_code,stop_name,stop_lat,stop_lon,next_stop_id
360556,886842,106,NaT,NaN,300,BLOOR-DANFORTH NIGHT BUS,EAST - 300B BLOOR-DANFORTH towards KENNEDY STA...,9949,1038,North Service Rd at Eglinton Ave East,43.733029,-79.265477,3169.0
361796,886852,1,0 days 00:02:38,158.000000,300,BLOOR-DANFORTH NIGHT BUS,WEST - 300A BLOOR-DANFORTH towards PEARSON AIR...,7221,14092,Pearson Airport Terminal 1 (Ground Level Colum...,43.681739,-79.611041,3169.0
365449,886878,94,0 days 00:02:41,161.000000,300,BLOOR-DANFORTH NIGHT BUS,WEST - 300A BLOOR-DANFORTH towards PEARSON AIR...,7221,14092,Pearson Airport Terminal 1 (Ground Level Colum...,43.681739,-79.611041,3169.0
414554,887071,12,NaT,NaN,33,FOREST HILL,NORTH - 33 FOREST HILL towards EGLINTON (ROSEL...,3129,2415,Eglinton Ave West at Vesta Dr,43.702097,-79.420708,3169.0
415365,887075,54,0 days 00:02:41,161.000000,332,EGLINTON WEST NIGHT BUS,WEST - 332 EGLINTON WEST towards RENFORTH AND ...,7221,14092,Pearson Airport Terminal 1 (Ground Level Colum...,43.681739,-79.611041,3169.0
415430,887076,54,0 days 00:07:10,430.000000,332,EGLINTON WEST NIGHT BUS,WEST - 332 EGLINTON WEST towards RENFORTH AND ...,7221,14092,Pearson Airport Terminal 1 (Ground Level Colum...,43.681739,-79.611041,3169.0
460847,887149,15,NaT,NaN,352,LAWRENCE WEST NIGHT BUS,EAST - 352 LAWRENCE WEST towards SUNNYBROOK HO...,854,1729,Hospital Rd at Wellness Way East Side (Sunnybr...,43.721780,-79.376853,3169.0
461683,887156,70,0 days 00:03:30,210.000000,352,LAWRENCE WEST NIGHT BUS,WEST - 352 LAWRENCE WEST towards PEARSON AIRPORT,7221,14092,Pearson Airport Terminal 1 (Ground Level Colum...,43.681739,-79.611041,3169.0
772381,887670,17,0 days 00:02:43,163.000000,52,LAWRENCE WEST,EAST - 52 LAWRENCE WEST towards LAWRENCE STATION,7221,14092,Pearson Airport Terminal 1 (Ground Level Colum...,43.681739,-79.611041,3169.0
782532,887687,72,NaT,NaN,52,LAWRENCE WEST,EAST - 52 LAWRENCE WEST towards LAWRENCE STATION,14244,14725,Lawrence Station,43.725556,-79.402351,3169.0


In [89]:
seq = seq[~seq.route_long_name.str.lower().str.contains('night')]
seq = seq[~seq.route_long_name.str.lower().str.contains('express')]

In [101]:
seq[seq.route_short_name > 100].route_short_name.value_counts().sort_index()

101     13
102    451
104     60
105     54
106     73
107    274
108    260
109    228
110    209
111     65
112    321
113     59
115     51
116    221
118    154
119     99
120    128
121     36
122    178
123    221
124     39
125     38
126     28
127     53
129    171
130    109
131    162
132     80
133    236
134    327
135     44
160    138
161     77
162     93
165    490
167    121
168     41
169    212
171     20
189     43
301    176
304    100
306    117
310     54
403     92
404     91
405    139
501    310
503    139
504    246
505    352
506    345
509     36
510     93
511    114
512    201
Name: route_short_name, dtype: int64

In [106]:
seq.loc[seq.route_short_name > 300, ['route_short_name', 'route_long_name']].value_counts().sort_index()

route_short_name  route_long_name
301               QUEEN              176
304               KING               100
306               CARLTON            117
310               SPADINA             54
403               DON MILLS SOUTH     92
404               EAST YORK           91
405               ETOBICOKE          139
501               QUEEN              310
503               KINGSTON RD        139
504               KING               246
505               DUNDAS             352
506               CARLTON            345
509               HARBOURFRONT        36
510               SPADINA             93
511               BATHURST           114
512               ST. CLAIR          201
dtype: int64

In [115]:
seq.loc[seq.route_short_name == 510, ['route_short_name', 'route_long_name', 'trip_headsign']].value_counts().sort_index()

route_short_name  route_long_name  trip_headsign                              
510               SPADINA          NORTH - 510 SPADINA SHORT TURN                  9
                                   NORTH - 510 SPADINA towards SPADINA STATION    43
                                   SOUTH - 510A SPADINA towards UNION STATION     17
                                   SOUTH - 510B SPADINA towards QUEENS QUAY       14
                                   SOUTH - 510C SPADINA towards KING              10
dtype: int64

In [120]:
seq.stop_id.unique()

array([14155,  3807,  6904, ..., 14537, 14538, 14539])

In [121]:
seq[seq.stop_id == 917]

,shape_id,stop_sequence,next_stop_duration,next_stop_seconds,route_short_name,route_long_name,trip_headsign,stop_id,stop_code,stop_name,stop_lat,stop_lon,next_stop_id
1187485,888274,48,0 days 00:00:31.500000,31.500000,95,YORK MILLS,WEST - 95 YORK MILLS towards YORK MILLS STATION,917,9083,York Mills Rd at Sandover Dr (1222 York Mills),43.759813,-79.331751,5191.0
1191050,888280,17,0 days 00:00:35.500000,35.500000,95,YORK MILLS,WEST - 95C YORK MILLS towards YORK MILLS STATION,917,9083,York Mills Rd at Sandover Dr (1222 York Mills),43.759813,-79.331751,5191.0
1193169,888281,39,0 days 00:00:34.666666666,34.666667,95,YORK MILLS,WEST - 95 YORK MILLS towards YORK MILLS STATION,917,9083,York Mills Rd at Sandover Dr (1222 York Mills),43.759813,-79.331751,5191.0


In [125]:
ym_shapes = seq[seq.shape_id.isin([888274, 888280, 888281])]

In [141]:
a = seq[seq.shape_id == 888274]
b = seq[seq.shape_id == 888280]
c = seq[seq.shape_id == 888281]

In [144]:
len(a), len(b), len(c)

(70, 39, 61)

In [156]:
a[a.stop_id == 2010]

,shape_id,stop_sequence,next_stop_duration,next_stop_seconds,route_short_name,route_long_name,trip_headsign,stop_id,stop_code,stop_name,stop_lat,stop_lon,next_stop_id
1185880,888274,33,0 days 00:01:06.500000,66.5,95,YORK MILLS,WEST - 95 YORK MILLS towards YORK MILLS STATION,2010,7692,Ellesmere Rd at Kennedy Rd,43.766017,-79.28076,1485.0


In [168]:
A = a[a.stop_sequence >= 33].stop_id.to_list()

In [165]:
B = b[b.stop_sequence > 1].stop_id.to_list()

In [169]:
A == B

True

In [172]:
a[a.stop_sequence >= 33]

,shape_id,stop_sequence,next_stop_duration,next_stop_seconds,route_short_name,route_long_name,trip_headsign,stop_id,stop_code,stop_name,stop_lat,stop_lon,next_stop_id
1185880,888274,33,0 days 00:01:06.500000,66.500000,95,YORK MILLS,WEST - 95 YORK MILLS towards YORK MILLS STATION,2010,7692,Ellesmere Rd at Kennedy Rd,43.766017,-79.280760,1485.0
1185987,888274,34,0 days 00:00:54,54.000000,95,YORK MILLS,WEST - 95 YORK MILLS towards YORK MILLS STATION,1485,7722,Ellesmere Rd at Dundalk Dr West Side,43.765185,-79.284336,8864.0
1186094,888274,35,0 days 00:01:16.250000,76.250000,95,YORK MILLS,WEST - 95 YORK MILLS towards YORK MILLS STATION,8864,14329,741 Ellesmere Rd at Mondeo Dr,43.764459,-79.287449,4604.0
1186201,888274,36,0 days 00:00:33,33.000000,95,YORK MILLS,WEST - 95 YORK MILLS towards YORK MILLS STATION,4604,7679,Ellesmere Rd at Birchmount Rd West Side,43.763409,-79.291907,6543.0
1186308,888274,37,0 days 00:00:36,36.000000,95,YORK MILLS,WEST - 95 YORK MILLS towards YORK MILLS STATION,6543,11859,Ellesmere Rd at Principal Rd,43.762790,-79.294575,8592.0
1186415,888274,38,0 days 00:00:36.333333333,36.333333,95,YORK MILLS,WEST - 95 YORK MILLS towards YORK MILLS STATION,8592,7708,426 Ellesmere Rd,43.762102,-79.297487,7360.0
1186522,888274,39,0 days 00:00:49,49.000000,95,YORK MILLS,WEST - 95 YORK MILLS towards YORK MILLS STATION,7360,7720,Ellesmere Rd at Warden Ave,43.761429,-79.300415,3350.0
1186629,888274,40,0 days 00:00:43,43.000000,95,YORK MILLS,WEST - 95 YORK MILLS towards YORK MILLS STATION,3350,7689,Ellesmere Rd at Crocus Dr,43.760452,-79.304350,14573.0
1186736,888274,41,0 days 00:00:27.666666666,27.666667,95,YORK MILLS,WEST - 95 YORK MILLS towards YORK MILLS STATION,14573,7686,Ellesmere Rd at Claudius Gate,43.759681,-79.307806,5470.0
1186843,888274,42,0 days 00:00:58,58.000000,95,YORK MILLS,WEST - 95 YORK MILLS towards YORK MILLS STATION,5470,7714,Ellesmere Rd at Pharmacy Ave,43.759180,-79.310072,353.0


In [174]:
c[c.stop_id == 2010]

,shape_id,stop_sequence,next_stop_duration,next_stop_seconds,route_short_name,route_long_name,trip_headsign,stop_id,stop_code,stop_name,stop_lat,stop_lon,next_stop_id
1192959,888281,24,0 days 00:01:06.250000,66.25,95,YORK MILLS,WEST - 95 YORK MILLS towards YORK MILLS STATION,2010,7692,Ellesmere Rd at Kennedy Rd,43.766017,-79.28076,1485.0


In [178]:
C = c[c.stop_sequence >= 24].stop_id.to_list()

In [180]:
B == C

True

In [181]:
c

,shape_id,stop_sequence,next_stop_duration,next_stop_seconds,route_short_name,route_long_name,trip_headsign,stop_id,stop_code,stop_name,stop_lat,stop_lon,next_stop_id
1192637,888281,1,0 days 00:00:41,41.0,95,YORK MILLS,WEST - 95 YORK MILLS towards YORK MILLS STATION,14923,15116,University of Toronto Scarborough,43.784391,-79.184989,6974.0
1192651,888281,2,0 days 00:00:28,28.0,95,YORK MILLS,WEST - 95 YORK MILLS towards YORK MILLS STATION,6974,7977,Military Trail at University of Toronto Scarbo...,43.785926,-79.186773,4049.0
1192665,888281,3,0 days 00:00:45,45.0,95,YORK MILLS,WEST - 95 YORK MILLS towards YORK MILLS STATION,4049,7699,Ellesmere Rd at Military Trail (East) West Side,43.786502,-79.189027,9612.0
1192679,888281,4,0 days 00:00:22.500000,22.5,95,YORK MILLS,WEST - 95 YORK MILLS towards YORK MILLS STATION,9612,7703,Ellesmere Rd at Morningside Ave,43.785523,-79.193129,1655.0
1192693,888281,5,0 days 00:01:45,105.0,95,YORK MILLS,WEST - 95 YORK MILLS towards YORK MILLS STATION,1655,7701,Ellesmere Rd at Mornelle Crt,43.785001,-79.195193,5200.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1193421,888281,57,0 days 00:00:22.500000,22.5,95,YORK MILLS,WEST - 95 YORK MILLS towards YORK MILLS STATION,1852,9074,York Mills Rd at Old Yonge St,43.744926,-79.399283,9572.0
1193435,888281,58,0 days 00:00:37,37.0,95,YORK MILLS,WEST - 95 YORK MILLS towards YORK MILLS STATION,9572,3265,York Mills Rd at York Ridge Rd,43.744470,-79.401416,6062.0
1193449,888281,59,0 days 00:00:10,10.0,95,YORK MILLS,WEST - 95 YORK MILLS towards YORK MILLS STATION,6062,11334,York Mills Rd at Old York Mills Rd,43.744461,-79.404964,919.0
1193463,888281,60,0 days 00:00:33,33.0,95,YORK MILLS,WEST - 95 YORK MILLS towards YORK MILLS STATION,919,9906,York Mills Rd at Yonge St,43.744417,-79.405968,24357.0


In [182]:
61-24

37

In [183]:
39-2

37

In [184]:
70-33

37

In [187]:
23+38

61

In [189]:
x = a[a.stop_sequence < 33]

In [191]:
y = c[c.stop_sequence < 24]

In [199]:
len(x), len(y)

(32, 23)

In [203]:
len(x[x.stop_id.isin(y.stop_id)].stop_id)

21

In [214]:
X = x[x.stop_id.isin(y.stop_id)].stop_id.to_list()

In [215]:
Y = y[y.stop_sequence > 2].stop_id.to_list()

In [216]:
X == Y

True

In [217]:
y

,shape_id,stop_sequence,next_stop_duration,next_stop_seconds,route_short_name,route_long_name,trip_headsign,stop_id,stop_code,stop_name,stop_lat,stop_lon,next_stop_id
1192637,888281,1,0 days 00:00:41,41.000000,95,YORK MILLS,WEST - 95 YORK MILLS towards YORK MILLS STATION,14923,15116,University of Toronto Scarborough,43.784391,-79.184989,6974.0
1192651,888281,2,0 days 00:00:28,28.000000,95,YORK MILLS,WEST - 95 YORK MILLS towards YORK MILLS STATION,6974,7977,Military Trail at University of Toronto Scarbo...,43.785926,-79.186773,4049.0
1192665,888281,3,0 days 00:00:45,45.000000,95,YORK MILLS,WEST - 95 YORK MILLS towards YORK MILLS STATION,4049,7699,Ellesmere Rd at Military Trail (East) West Side,43.786502,-79.189027,9612.0
1192679,888281,4,0 days 00:00:22.500000,22.500000,95,YORK MILLS,WEST - 95 YORK MILLS towards YORK MILLS STATION,9612,7703,Ellesmere Rd at Morningside Ave,43.785523,-79.193129,1655.0
1192693,888281,5,0 days 00:01:45,105.000000,95,YORK MILLS,WEST - 95 YORK MILLS towards YORK MILLS STATION,1655,7701,Ellesmere Rd at Mornelle Crt,43.785001,-79.195193,5200.0
1192707,888281,6,0 days 00:02:15,135.000000,95,YORK MILLS,WEST - 95 YORK MILLS towards YORK MILLS STATION,5200,7705,Ellesmere Rd at Neilson Rd (Scarborough Centen...,43.782805,-79.204762,6004.0
1192721,888281,7,0 days 00:01:01.250000,61.250000,95,YORK MILLS,WEST - 95 YORK MILLS towards YORK MILLS STATION,6004,7700,Ellesmere Rd at Military Trail (West),43.779894,-79.217004,726.0
1192735,888281,8,0 days 00:00:47,47.000000,95,YORK MILLS,WEST - 95 YORK MILLS towards YORK MILLS STATION,726,7691,Ellesmere Rd at Helicon Gate West Side,43.778605,-79.222594,7343.0
1192749,888281,9,0 days 00:00:47,47.000000,95,YORK MILLS,WEST - 95 YORK MILLS towards YORK MILLS STATION,7343,1045,Ellesmere Rd at Dormington Dr West Side,43.777621,-79.226939,1104.0
1192763,888281,10,0 days 00:00:55,55.000000,95,YORK MILLS,WEST - 95 YORK MILLS towards YORK MILLS STATION,1104,1049,Ellesmere Rd at Markham Rd,43.776669,-79.231287,7874.0


In [219]:
y[y.stop_sequence < 3]

,shape_id,stop_sequence,next_stop_duration,next_stop_seconds,route_short_name,route_long_name,trip_headsign,stop_id,stop_code,stop_name,stop_lat,stop_lon,next_stop_id
1192637,888281,1,0 days 00:00:41,41.0,95,YORK MILLS,WEST - 95 YORK MILLS towards YORK MILLS STATION,14923,15116,University of Toronto Scarborough,43.784391,-79.184989,6974.0
1192651,888281,2,0 days 00:00:28,28.0,95,YORK MILLS,WEST - 95 YORK MILLS towards YORK MILLS STATION,6974,7977,Military Trail at University of Toronto Scarbo...,43.785926,-79.186773,4049.0


In [222]:
x[x.stop_sequence < 12]

,shape_id,stop_sequence,next_stop_duration,next_stop_seconds,route_short_name,route_long_name,trip_headsign,stop_id,stop_code,stop_name,stop_lat,stop_lon,next_stop_id
1182456,888274,1,0 days 00:00:30.500000,30.500000,95,YORK MILLS,WEST - 95 YORK MILLS towards YORK MILLS STATION,10122,10521,Cul De Sac (Loop) at Ellesmere Rd,43.794687,-79.154799,3892.0
1182563,888274,2,0 days 00:00:58.500000,58.500000,95,YORK MILLS,WEST - 95 YORK MILLS towards YORK MILLS STATION,3892,10522,Ellesmere Rd at Muirbank Blvd,43.793969,-79.157635,4388.0
1182670,888274,3,0 days 00:00:41.750000,41.750000,95,YORK MILLS,WEST - 95 YORK MILLS towards YORK MILLS STATION,4388,10523,Ellesmere Rd at Meadowvale Rd,43.792623,-79.163292,7788.0
1182777,888274,4,0 days 00:00:39,39.000000,95,YORK MILLS,WEST - 95 YORK MILLS towards YORK MILLS STATION,7788,10524,Ellesmere Rd at Zaph Ave,43.791840,-79.166565,4807.0
1182884,888274,5,0 days 00:00:40,40.000000,95,YORK MILLS,WEST - 95 YORK MILLS towards YORK MILLS STATION,4807,10525,Ellesmere Rd at Scarboro Ave,43.791060,-79.169811,7828.0
1182991,888274,6,0 days 00:00:41.333333333,41.333333,95,YORK MILLS,WEST - 95 YORK MILLS towards YORK MILLS STATION,7828,10526,Ellesmere Rd at Morrish Rd,43.790274,-79.173121,1689.0
1183098,888274,7,0 days 00:00:26.333333333,26.333333,95,YORK MILLS,WEST - 95 YORK MILLS towards YORK MILLS STATION,1689,10527,Ellesmere Rd at Calverley Trail,43.789490,-79.176571,8793.0
1183205,888274,8,0 days 00:00:20,20.000000,95,YORK MILLS,WEST - 95 YORK MILLS towards YORK MILLS STATION,8793,10528,Ellesmere Rd at Highland Creek Park,43.788976,-79.178809,8302.0
1183312,888274,9,0 days 00:00:29,29.000000,95,YORK MILLS,WEST - 95 YORK MILLS towards YORK MILLS STATION,8302,9471,Ellesmere Rd at Calthorpe Ave,43.788586,-79.180524,9326.0
1183419,888274,10,0 days 00:01:03,63.000000,95,YORK MILLS,WEST - 95 YORK MILLS towards YORK MILLS STATION,9326,9472,Ellesmere Rd at Conlins Rd,43.788024,-79.182973,7684.0


In [223]:
b.head(1)

,shape_id,stop_sequence,next_stop_duration,next_stop_seconds,route_short_name,route_long_name,trip_headsign,stop_id,stop_code,stop_name,stop_lat,stop_lon,next_stop_id
1189946,888280,1,0 days 00:00:46,46.0,95,YORK MILLS,WEST - 95C YORK MILLS towards YORK MILLS STATION,15015,15189,Ellesmere Station,43.766958,-79.277486,2010.0


In [224]:
a[a.stop_sequence > 11]

,shape_id,stop_sequence,next_stop_duration,next_stop_seconds,route_short_name,route_long_name,trip_headsign,stop_id,stop_code,stop_name,stop_lat,stop_lon,next_stop_id
1183633,888274,12,0 days 00:00:45,45.000000,95,YORK MILLS,WEST - 95 YORK MILLS towards YORK MILLS STATION,4049,7699,Ellesmere Rd at Military Trail (East) West Side,43.786502,-79.189027,9612.0
1183740,888274,13,0 days 00:00:22.500000,22.500000,95,YORK MILLS,WEST - 95 YORK MILLS towards YORK MILLS STATION,9612,7703,Ellesmere Rd at Morningside Ave,43.785523,-79.193129,1655.0
1183847,888274,14,0 days 00:01:45,105.000000,95,YORK MILLS,WEST - 95 YORK MILLS towards YORK MILLS STATION,1655,7701,Ellesmere Rd at Mornelle Crt,43.785001,-79.195193,5200.0
1183954,888274,15,0 days 00:02:15,135.000000,95,YORK MILLS,WEST - 95 YORK MILLS towards YORK MILLS STATION,5200,7705,Ellesmere Rd at Neilson Rd (Scarborough Centen...,43.782805,-79.204762,6004.0
1184061,888274,16,0 days 00:01:01.250000,61.250000,95,YORK MILLS,WEST - 95 YORK MILLS towards YORK MILLS STATION,6004,7700,Ellesmere Rd at Military Trail (West),43.779894,-79.217004,726.0
1184168,888274,17,0 days 00:00:47,47.000000,95,YORK MILLS,WEST - 95 YORK MILLS towards YORK MILLS STATION,726,7691,Ellesmere Rd at Helicon Gate West Side,43.778605,-79.222594,7343.0
1184275,888274,18,0 days 00:00:47,47.000000,95,YORK MILLS,WEST - 95 YORK MILLS towards YORK MILLS STATION,7343,1045,Ellesmere Rd at Dormington Dr West Side,43.777621,-79.226939,1104.0
1184382,888274,19,0 days 00:01:03.333333333,63.333333,95,YORK MILLS,WEST - 95 YORK MILLS towards YORK MILLS STATION,1104,1049,Ellesmere Rd at Markham Rd,43.776669,-79.231287,7874.0
1184489,888274,20,0 days 00:01:32.333333333,92.333333,95,YORK MILLS,WEST - 95 YORK MILLS towards YORK MILLS STATION,7874,1043,Ellesmere Rd at Dolly Varden Blvd West Side,43.775797,-79.235291,6990.0
1184596,888274,21,0 days 00:01:06.666666666,66.666667,95,YORK MILLS,WEST - 95 YORK MILLS towards YORK MILLS STATION,6990,7677,Ellesmere Rd at Bellamy Rd North,43.774537,-79.241437,5002.0


In [229]:
seq.loc[seq.route_short_name == 95, ['shape_id', 'trip_headsign']].value_counts().sort_index()

shape_id  trip_headsign                                      
888270    EAST - 95C YORK MILLS towards ELLESMERE STATION        36
888271    EAST - 95B YORK MILLS towards U OF T SCARBOROUGH       57
888272    EAST - 95A YORK MILLS towards PORT UNION               68
888273    EAST - 95A YORK MILLS towards PORT UNION                5
888274    WEST - 95 YORK MILLS towards YORK MILLS STATION        70
888280    WEST - 95C YORK MILLS towards YORK MILLS STATION       39
888281    WEST - 95 YORK MILLS towards YORK MILLS STATION        61
888282    WEST - 95S YORK MILLS SHORT TURN towards MEADOWVALE     3
dtype: int64

In [232]:
seq.loc[seq.route_short_name == 95].shape_id.value_counts().sort_index()

888270    36
888271    57
888272    68
888273     5
888274    70
888280    39
888281    61
888282     3
Name: shape_id, dtype: int64

In [233]:
seq[seq.shape_id == 888282]

,shape_id,stop_sequence,next_stop_duration,next_stop_seconds,route_short_name,route_long_name,trip_headsign,stop_id,stop_code,stop_name,stop_lat,stop_lon,next_stop_id
1193491,888282,1,0 days 00:00:25,25.0,95,YORK MILLS,WEST - 95S YORK MILLS SHORT TURN towards MEADO...,10122,10521,Cul De Sac (Loop) at Ellesmere Rd,43.794687,-79.154799,3892.0
1193493,888282,2,0 days 00:00:47,47.0,95,YORK MILLS,WEST - 95S YORK MILLS SHORT TURN towards MEADO...,3892,10522,Ellesmere Rd at Muirbank Blvd,43.793969,-79.157635,4388.0
1193495,888282,3,NaT,NaN,95,YORK MILLS,WEST - 95S YORK MILLS SHORT TURN towards MEADO...,4388,10523,Ellesmere Rd at Meadowvale Rd,43.792623,-79.163292,3169.0


In [234]:
a

,shape_id,stop_sequence,next_stop_duration,next_stop_seconds,route_short_name,route_long_name,trip_headsign,stop_id,stop_code,stop_name,stop_lat,stop_lon,next_stop_id
1182456,888274,1,0 days 00:00:30.500000,30.500000,95,YORK MILLS,WEST - 95 YORK MILLS towards YORK MILLS STATION,10122,10521,Cul De Sac (Loop) at Ellesmere Rd,43.794687,-79.154799,3892.0
1182563,888274,2,0 days 00:00:58.500000,58.500000,95,YORK MILLS,WEST - 95 YORK MILLS towards YORK MILLS STATION,3892,10522,Ellesmere Rd at Muirbank Blvd,43.793969,-79.157635,4388.0
1182670,888274,3,0 days 00:00:41.750000,41.750000,95,YORK MILLS,WEST - 95 YORK MILLS towards YORK MILLS STATION,4388,10523,Ellesmere Rd at Meadowvale Rd,43.792623,-79.163292,7788.0
1182777,888274,4,0 days 00:00:39,39.000000,95,YORK MILLS,WEST - 95 YORK MILLS towards YORK MILLS STATION,7788,10524,Ellesmere Rd at Zaph Ave,43.791840,-79.166565,4807.0
1182884,888274,5,0 days 00:00:40,40.000000,95,YORK MILLS,WEST - 95 YORK MILLS towards YORK MILLS STATION,4807,10525,Ellesmere Rd at Scarboro Ave,43.791060,-79.169811,7828.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1189411,888274,66,0 days 00:00:20,20.000000,95,YORK MILLS,WEST - 95 YORK MILLS towards YORK MILLS STATION,1852,9074,York Mills Rd at Old Yonge St,43.744926,-79.399283,9572.0
1189518,888274,67,0 days 00:00:33,33.000000,95,YORK MILLS,WEST - 95 YORK MILLS towards YORK MILLS STATION,9572,3265,York Mills Rd at York Ridge Rd,43.744470,-79.401416,6062.0
1189625,888274,68,0 days 00:00:08.666666666,8.666667,95,YORK MILLS,WEST - 95 YORK MILLS towards YORK MILLS STATION,6062,11334,York Mills Rd at Old York Mills Rd,43.744461,-79.404964,919.0
1189732,888274,69,0 days 00:00:29.333333333,29.333333,95,YORK MILLS,WEST - 95 YORK MILLS towards YORK MILLS STATION,919,9906,York Mills Rd at Yonge St,43.744417,-79.405968,24357.0


In [237]:
a

,shape_id,stop_sequence,next_stop_duration,next_stop_seconds,route_short_name,route_long_name,trip_headsign,stop_id,stop_code,stop_name,stop_lat,stop_lon,next_stop_id
1182456,888274,1,0 days 00:00:30.500000,30.500000,95,YORK MILLS,WEST - 95 YORK MILLS towards YORK MILLS STATION,10122,10521,Cul De Sac (Loop) at Ellesmere Rd,43.794687,-79.154799,3892.0
1182563,888274,2,0 days 00:00:58.500000,58.500000,95,YORK MILLS,WEST - 95 YORK MILLS towards YORK MILLS STATION,3892,10522,Ellesmere Rd at Muirbank Blvd,43.793969,-79.157635,4388.0
1182670,888274,3,0 days 00:00:41.750000,41.750000,95,YORK MILLS,WEST - 95 YORK MILLS towards YORK MILLS STATION,4388,10523,Ellesmere Rd at Meadowvale Rd,43.792623,-79.163292,7788.0
1182777,888274,4,0 days 00:00:39,39.000000,95,YORK MILLS,WEST - 95 YORK MILLS towards YORK MILLS STATION,7788,10524,Ellesmere Rd at Zaph Ave,43.791840,-79.166565,4807.0
1182884,888274,5,0 days 00:00:40,40.000000,95,YORK MILLS,WEST - 95 YORK MILLS towards YORK MILLS STATION,4807,10525,Ellesmere Rd at Scarboro Ave,43.791060,-79.169811,7828.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1189411,888274,66,0 days 00:00:20,20.000000,95,YORK MILLS,WEST - 95 YORK MILLS towards YORK MILLS STATION,1852,9074,York Mills Rd at Old Yonge St,43.744926,-79.399283,9572.0
1189518,888274,67,0 days 00:00:33,33.000000,95,YORK MILLS,WEST - 95 YORK MILLS towards YORK MILLS STATION,9572,3265,York Mills Rd at York Ridge Rd,43.744470,-79.401416,6062.0
1189625,888274,68,0 days 00:00:08.666666666,8.666667,95,YORK MILLS,WEST - 95 YORK MILLS towards YORK MILLS STATION,6062,11334,York Mills Rd at Old York Mills Rd,43.744461,-79.404964,919.0
1189732,888274,69,0 days 00:00:29.333333333,29.333333,95,YORK MILLS,WEST - 95 YORK MILLS towards YORK MILLS STATION,919,9906,York Mills Rd at Yonge St,43.744417,-79.405968,24357.0


In [239]:
seq[seq.stop_name.str.lower().str.contains('port union')]

,shape_id,stop_sequence,next_stop_duration,next_stop_seconds,route_short_name,route_long_name,trip_headsign,stop_id,stop_code,stop_name,stop_lat,stop_lon,next_stop_id
504346,887240,3,0 days 00:00:22.500000,22.500000,38,HIGHLAND CREEK,WEST - 38 HIGHLAND CREEK towards SCARBOROUGH C...,6640,10141,Port Union Rd at Lawrence Ave East North Side,43.780021,-79.136654,620.0
504435,887240,4,0 days 00:00:44.500000,44.500000,38,HIGHLAND CREEK,WEST - 38 HIGHLAND CREEK towards SCARBOROUGH C...,620,9885,Port Union Rd at Adenmore Rd North Side,43.781768,-79.137470,3888.0
504524,887240,5,0 days 00:00:40.500000,40.500000,38,HIGHLAND CREEK,WEST - 38 HIGHLAND CREEK towards SCARBOROUGH C...,3888,9886,Port Union Rd at Winter Gardens Trail,43.785227,-79.138910,10110.0
504613,887240,6,0 days 00:00:44.250000,44.250000,38,HIGHLAND CREEK,WEST - 38 HIGHLAND CREEK towards SCARBOROUGH C...,10110,9887,Port Union Rd at Ravine Park Cres North Side,43.788351,-79.140204,8584.0
507289,887241,3,0 days 00:00:17,17.000000,38,HIGHLAND CREEK,WEST - 38S HIGHLAND CREEK SHORT TURN towards M...,6640,10141,Port Union Rd at Lawrence Ave East North Side,43.780021,-79.136654,620.0
507292,887241,4,0 days 00:00:34,34.000000,38,HIGHLAND CREEK,WEST - 38S HIGHLAND CREEK SHORT TURN towards M...,620,9885,Port Union Rd at Adenmore Rd North Side,43.781768,-79.137470,3888.0
507295,887241,5,0 days 00:00:31,31.000000,38,HIGHLAND CREEK,WEST - 38S HIGHLAND CREEK SHORT TURN towards M...,3888,9886,Port Union Rd at Winter Gardens Trail,43.785227,-79.138910,10110.0
507298,887241,6,0 days 00:00:34,34.000000,38,HIGHLAND CREEK,WEST - 38S HIGHLAND CREEK SHORT TURN towards M...,10110,9887,Port Union Rd at Ravine Park Cres North Side,43.788351,-79.140204,8584.0
510584,887243,30,0 days 00:00:29.600000,29.600000,38,HIGHLAND CREEK,EAST - 38A HIGHLAND CREEK towards ROUGE HILL G...,5743,7934,Lawson Rd at Port Union Rd,43.789502,-79.141184,4595.0
510673,887243,31,0 days 00:00:22.600000,22.600000,38,HIGHLAND CREEK,EAST - 38A HIGHLAND CREEK towards ROUGE HILL G...,4595,8037,Port Union Rd at Ravine Park Cres South Side,43.787808,-79.140236,2545.0


In [276]:
d = seq[seq.shape_id == 888282]
e = seq[seq.shape_id == 888270]
f = seq[seq.shape_id == 888271]
g = seq[seq.shape_id == 888272]
h = seq[seq.shape_id == 888273]

In [258]:
h.stop_id.isin(g.stop_id)

1182411    True
1182420    True
1182429    True
1182438    True
1182447    True
Name: stop_id, dtype: bool

In [260]:
g[g.stop_id == 15015]

,shape_id,stop_sequence,next_stop_duration,next_stop_seconds,route_short_name,route_long_name,trip_headsign,stop_id,stop_code,stop_name,stop_lat,stop_lon,next_stop_id


In [262]:
seq[seq.stop_id == 15015]

,shape_id,stop_sequence,next_stop_duration,next_stop_seconds,route_short_name,route_long_name,trip_headsign,stop_id,stop_code,stop_name,stop_lat,stop_lon,next_stop_id
1174866,888270,36,NaT,NaN,95,YORK MILLS,EAST - 95C YORK MILLS towards ELLESMERE STATION,15015,15189,Ellesmere Station,43.766958,-79.277486,24357.0
1189946,888280,1,0 days 00:00:46,46.0,95,YORK MILLS,WEST - 95C YORK MILLS towards YORK MILLS STATION,15015,15189,Ellesmere Station,43.766958,-79.277486,2010.0


In [265]:
g[~g.stop_id.isin(f.stop_id)]

,shape_id,stop_sequence,next_stop_duration,next_stop_seconds,route_short_name,route_long_name,trip_headsign,stop_id,stop_code,stop_name,stop_lat,stop_lon,next_stop_id
1181211,888272,57,0 days 00:00:24.666666666,24.666667,95,YORK MILLS,EAST - 95A YORK MILLS towards PORT UNION,4356,9468,Ellesmere Rd at Conlins Rd,43.787692,-79.183514,9246.0
1181311,888272,58,0 days 00:00:16.666666666,16.666667,95,YORK MILLS,EAST - 95A YORK MILLS towards PORT UNION,9246,9469,Ellesmere Rd at Calthorpe Ave,43.788307,-79.180898,4982.0
1181411,888272,59,0 days 00:00:31,31.000000,95,YORK MILLS,EAST - 95A YORK MILLS towards PORT UNION,4982,10514,3507 Ellesmere Rd,43.788712,-79.179141,4051.0
1181511,888272,60,0 days 00:00:21.333333333,21.333333,95,YORK MILLS,EAST - 95A YORK MILLS towards PORT UNION,4051,10515,Ellesmere Rd at Watson St,43.789459,-79.175830,3076.0
1181611,888272,61,0 days 00:00:30.333333333,30.333333,95,YORK MILLS,EAST - 95A YORK MILLS towards PORT UNION,3076,10516,Ellesmere Rd at Morrish Rd,43.789964,-79.173574,4881.0
1181711,888272,62,0 days 00:00:35,35.000000,95,YORK MILLS,EAST - 95A YORK MILLS towards PORT UNION,4881,10517,Ellesmere Rd at Deep Dene Dr,43.790749,-79.170421,3087.0
1181811,888272,63,0 days 00:00:27,27.000000,95,YORK MILLS,EAST - 95A YORK MILLS towards PORT UNION,3087,10518,Ellesmere Rd at Zaph Ave,43.791626,-79.166730,10351.0
1181911,888272,64,0 days 00:02:27.500000,147.500000,95,YORK MILLS,EAST - 95A YORK MILLS towards PORT UNION,10351,10519,Ellesmere Rd at Meadowvale Rd,43.792312,-79.163843,9463.0
1182011,888272,65,0 days 00:01:49,109.000000,95,YORK MILLS,EAST - 95A YORK MILLS towards PORT UNION,9463,10520,Ellesmere Rd at Muirbank Blvd,43.793732,-79.157941,10122.0
1182111,888272,66,0 days 00:03:46,226.000000,95,YORK MILLS,EAST - 95A YORK MILLS towards PORT UNION,10122,10521,Cul De Sac (Loop) at Ellesmere Rd,43.794687,-79.154799,24410.0


In [266]:
h

,shape_id,stop_sequence,next_stop_duration,next_stop_seconds,route_short_name,route_long_name,trip_headsign,stop_id,stop_code,stop_name,stop_lat,stop_lon,next_stop_id
1182411,888273,1,0 days 00:02:46,166.0,95,YORK MILLS,EAST - 95A YORK MILLS towards PORT UNION,10351,10519,Ellesmere Rd at Meadowvale Rd,43.792312,-79.163843,9463.0
1182420,888273,2,0 days 00:01:49,109.0,95,YORK MILLS,EAST - 95A YORK MILLS towards PORT UNION,9463,10520,Ellesmere Rd at Muirbank Blvd,43.793732,-79.157941,10122.0
1182429,888273,3,0 days 00:03:46,226.0,95,YORK MILLS,EAST - 95A YORK MILLS towards PORT UNION,10122,10521,Cul De Sac (Loop) at Ellesmere Rd,43.794687,-79.154799,24410.0
1182438,888273,4,0 days 00:06:08,368.0,95,YORK MILLS,EAST - 95A YORK MILLS towards PORT UNION,24410,16023,Kingston Rd at Rylander Blvd,43.796887,-79.149251,1002.0
1182447,888273,5,NaT,NaN,95,YORK MILLS,EAST - 95A YORK MILLS towards PORT UNION,1002,7198,Sheppard Ave East at Kingston Rd North Side,43.799459,-79.146775,10122.0


In [268]:
f

,shape_id,stop_sequence,next_stop_duration,next_stop_seconds,route_short_name,route_long_name,trip_headsign,stop_id,stop_code,stop_name,stop_lat,stop_lon,next_stop_id
1174927,888271,1,0 days 00:00:39,39.0,95,YORK MILLS,EAST - 95B YORK MILLS towards U OF T SCARBOROUGH,24357,15974,York Mills Temporary Bus Loop,43.744303,-79.407324,2025.0
1174939,888271,2,0 days 00:00:39,39.0,95,YORK MILLS,EAST - 95B YORK MILLS towards U OF T SCARBOROUGH,2025,11333,York Mills Rd at Old York Mills Rd,43.744177,-79.404398,1767.0
1174951,888271,3,0 days 00:00:15,15.0,95,YORK MILLS,EAST - 95B YORK MILLS towards U OF T SCARBOROUGH,1767,3260,York Mills Rd at Hedgewood Rd East Side,43.744875,-79.398732,1096.0
1174963,888271,4,0 days 00:00:19,19.0,95,YORK MILLS,EAST - 95B YORK MILLS towards U OF T SCARBOROUGH,1096,3262,York Mills Rd at Highland Cres,43.745354,-79.396532,4905.0
1174975,888271,5,0 days 00:00:12,12.0,95,YORK MILLS,EAST - 95B YORK MILLS towards U OF T SCARBOROUGH,4905,3254,York Mills Rd at Cedarwood Ave East Side,43.745930,-79.393792,9374.0
1174987,888271,6,0 days 00:00:20,20.0,95,YORK MILLS,EAST - 95B YORK MILLS towards U OF T SCARBOROUGH,9374,3252,York Mills Rd at Birchwood Ave,43.746315,-79.392007,9893.0
1174999,888271,7,0 days 00:00:18,18.0,95,YORK MILLS,EAST - 95B YORK MILLS towards U OF T SCARBOROUGH,9893,3257,York Mills Rd at Fenn Ave,43.746927,-79.389067,4244.0
1175011,888271,8,0 days 00:00:13,13.0,95,YORK MILLS,EAST - 95B YORK MILLS towards U OF T SCARBOROUGH,4244,3259,York Mills Rd at Glenridge Ave,43.747477,-79.386471,1524.0
1175023,888271,9,0 days 00:01:00,60.0,95,YORK MILLS,EAST - 95B YORK MILLS towards U OF T SCARBOROUGH,1524,9188,York Mills Rd at Bayview Ave,43.747870,-79.384519,7488.0
1175035,888271,10,0 days 00:00:37,37.0,95,YORK MILLS,EAST - 95B YORK MILLS towards U OF T SCARBOROUGH,7488,7468,York Mills Rd at Harrison Rd,43.749601,-79.378889,4115.0


In [269]:
56-35

21

In [274]:
35+21+12

68

In [275]:
68-57+1

12

In [277]:
d

,shape_id,stop_sequence,next_stop_duration,next_stop_seconds,route_short_name,route_long_name,trip_headsign,stop_id,stop_code,stop_name,stop_lat,stop_lon,next_stop_id
1193491,888282,1,0 days 00:00:25,25.0,95,YORK MILLS,WEST - 95S YORK MILLS SHORT TURN towards MEADO...,10122,10521,Cul De Sac (Loop) at Ellesmere Rd,43.794687,-79.154799,3892.0
1193493,888282,2,0 days 00:00:47,47.0,95,YORK MILLS,WEST - 95S YORK MILLS SHORT TURN towards MEADO...,3892,10522,Ellesmere Rd at Muirbank Blvd,43.793969,-79.157635,4388.0
1193495,888282,3,NaT,NaN,95,YORK MILLS,WEST - 95S YORK MILLS SHORT TURN towards MEADO...,4388,10523,Ellesmere Rd at Meadowvale Rd,43.792623,-79.163292,3169.0


In [281]:
a.head(5)

,shape_id,stop_sequence,next_stop_duration,next_stop_seconds,route_short_name,route_long_name,trip_headsign,stop_id,stop_code,stop_name,stop_lat,stop_lon,next_stop_id
1182456,888274,1,0 days 00:00:30.500000,30.50,95,YORK MILLS,WEST - 95 YORK MILLS towards YORK MILLS STATION,10122,10521,Cul De Sac (Loop) at Ellesmere Rd,43.794687,-79.154799,3892.0
1182563,888274,2,0 days 00:00:58.500000,58.50,95,YORK MILLS,WEST - 95 YORK MILLS towards YORK MILLS STATION,3892,10522,Ellesmere Rd at Muirbank Blvd,43.793969,-79.157635,4388.0
1182670,888274,3,0 days 00:00:41.750000,41.75,95,YORK MILLS,WEST - 95 YORK MILLS towards YORK MILLS STATION,4388,10523,Ellesmere Rd at Meadowvale Rd,43.792623,-79.163292,7788.0
1182777,888274,4,0 days 00:00:39,39.00,95,YORK MILLS,WEST - 95 YORK MILLS towards YORK MILLS STATION,7788,10524,Ellesmere Rd at Zaph Ave,43.791840,-79.166565,4807.0
1182884,888274,5,0 days 00:00:40,40.00,95,YORK MILLS,WEST - 95 YORK MILLS towards YORK MILLS STATION,4807,10525,Ellesmere Rd at Scarboro Ave,43.791060,-79.169811,7828.0


In [282]:
h

,shape_id,stop_sequence,next_stop_duration,next_stop_seconds,route_short_name,route_long_name,trip_headsign,stop_id,stop_code,stop_name,stop_lat,stop_lon,next_stop_id
1182411,888273,1,0 days 00:02:46,166.0,95,YORK MILLS,EAST - 95A YORK MILLS towards PORT UNION,10351,10519,Ellesmere Rd at Meadowvale Rd,43.792312,-79.163843,9463.0
1182420,888273,2,0 days 00:01:49,109.0,95,YORK MILLS,EAST - 95A YORK MILLS towards PORT UNION,9463,10520,Ellesmere Rd at Muirbank Blvd,43.793732,-79.157941,10122.0
1182429,888273,3,0 days 00:03:46,226.0,95,YORK MILLS,EAST - 95A YORK MILLS towards PORT UNION,10122,10521,Cul De Sac (Loop) at Ellesmere Rd,43.794687,-79.154799,24410.0
1182438,888273,4,0 days 00:06:08,368.0,95,YORK MILLS,EAST - 95A YORK MILLS towards PORT UNION,24410,16023,Kingston Rd at Rylander Blvd,43.796887,-79.149251,1002.0
1182447,888273,5,NaT,NaN,95,YORK MILLS,EAST - 95A YORK MILLS towards PORT UNION,1002,7198,Sheppard Ave East at Kingston Rd North Side,43.799459,-79.146775,10122.0


In [283]:
g.tail(10)

,shape_id,stop_sequence,next_stop_duration,next_stop_seconds,route_short_name,route_long_name,trip_headsign,stop_id,stop_code,stop_name,stop_lat,stop_lon,next_stop_id
1181411,888272,59,0 days 00:00:31,31.000000,95,YORK MILLS,EAST - 95A YORK MILLS towards PORT UNION,4982,10514,3507 Ellesmere Rd,43.788712,-79.179141,4051.0
1181511,888272,60,0 days 00:00:21.333333333,21.333333,95,YORK MILLS,EAST - 95A YORK MILLS towards PORT UNION,4051,10515,Ellesmere Rd at Watson St,43.789459,-79.175830,3076.0
1181611,888272,61,0 days 00:00:30.333333333,30.333333,95,YORK MILLS,EAST - 95A YORK MILLS towards PORT UNION,3076,10516,Ellesmere Rd at Morrish Rd,43.789964,-79.173574,4881.0
1181711,888272,62,0 days 00:00:35,35.000000,95,YORK MILLS,EAST - 95A YORK MILLS towards PORT UNION,4881,10517,Ellesmere Rd at Deep Dene Dr,43.790749,-79.170421,3087.0
1181811,888272,63,0 days 00:00:27,27.000000,95,YORK MILLS,EAST - 95A YORK MILLS towards PORT UNION,3087,10518,Ellesmere Rd at Zaph Ave,43.791626,-79.166730,10351.0
1181911,888272,64,0 days 00:02:27.500000,147.500000,95,YORK MILLS,EAST - 95A YORK MILLS towards PORT UNION,10351,10519,Ellesmere Rd at Meadowvale Rd,43.792312,-79.163843,9463.0
1182011,888272,65,0 days 00:01:49,109.000000,95,YORK MILLS,EAST - 95A YORK MILLS towards PORT UNION,9463,10520,Ellesmere Rd at Muirbank Blvd,43.793732,-79.157941,10122.0
1182111,888272,66,0 days 00:03:46,226.000000,95,YORK MILLS,EAST - 95A YORK MILLS towards PORT UNION,10122,10521,Cul De Sac (Loop) at Ellesmere Rd,43.794687,-79.154799,24410.0
1182211,888272,67,0 days 00:06:08,368.000000,95,YORK MILLS,EAST - 95A YORK MILLS towards PORT UNION,24410,16023,Kingston Rd at Rylander Blvd,43.796887,-79.149251,1002.0
1182311,888272,68,NaT,NaN,95,YORK MILLS,EAST - 95A YORK MILLS towards PORT UNION,1002,7198,Sheppard Ave East at Kingston Rd North Side,43.799459,-79.146775,10351.0


In [284]:
schedule

,trip_id,route_short_name,route_long_name,shape_id,trip_headsign,stop_sequence,stop_time,stop_time_delta,stop_id,stop_code,stop_name,stop_lat,stop_lon
0,42990004,10,VAN HORNE,886387,EAST - 10 VAN HORNE towards VICTORIA PARK,1,7:00:29,0 days 07:00:29,14155,14633,Don Mills Station,43.776222,-79.347048
1,42990004,10,VAN HORNE,886387,EAST - 10 VAN HORNE towards VICTORIA PARK,2,7:01:19,0 days 07:01:19,3807,1949,Don Mills Rd at Leith Hill Rd North Side,43.777534,-79.347811
2,42990004,10,VAN HORNE,886387,EAST - 10 VAN HORNE towards VICTORIA PARK,3,7:02:12,0 days 07:02:12,6904,1929,Don Mills Rd at Fairview Mall Dr North Side,43.779530,-79.348701
3,42990004,10,VAN HORNE,886387,EAST - 10 VAN HORNE towards VICTORIA PARK,4,7:03:35,0 days 07:03:35,1163,1938,Don Mills Rd at Godstone Rd,43.782682,-79.348922
4,42990004,10,VAN HORNE,886387,EAST - 10 VAN HORNE towards VICTORIA PARK,5,7:04:48,0 days 07:04:48,7723,1919,Don Mills Rd at Deerford Rd,43.785281,-79.350570
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1304777,43128797,4,LINE 4 (SHEPPARD),890357,LINE 4 (SHEPPARD) towards SHEPPARD-YONGE STATION,1,5:31:00,0 days 05:31:00,14535,14109,Don Mills Station - Westbound Platform,43.775248,-79.346189
1304778,43128797,4,LINE 4 (SHEPPARD),890357,LINE 4 (SHEPPARD) towards SHEPPARD-YONGE STATION,2,5:33:05,0 days 05:33:05,14536,13847,Leslie Station - Westbound Platform,43.771248,-79.366790
1304779,43128797,4,LINE 4 (SHEPPARD),890357,LINE 4 (SHEPPARD) towards SHEPPARD-YONGE STATION,3,5:34:50,0 days 05:34:50,14537,13846,Bessarion Station - Westbound Platform,43.769296,-79.376345
1304780,43128797,4,LINE 4 (SHEPPARD),890357,LINE 4 (SHEPPARD) towards SHEPPARD-YONGE STATION,4,5:36:43,0 days 05:36:43,14538,13843,Bayview Station - Westbound Platform,43.766951,-79.386730


In [285]:
schedule[schedule.shape_id == 888282]

,trip_id,route_short_name,route_long_name,shape_id,trip_headsign,stop_sequence,stop_time,stop_time_delta,stop_id,stop_code,stop_name,stop_lat,stop_lon
1193491,43027291,95,YORK MILLS,888282,WEST - 95S YORK MILLS SHORT TURN towards MEADO...,1,26:00:35,1 days 02:00:35,10122,10521,Cul De Sac (Loop) at Ellesmere Rd,43.794687,-79.154799
1193492,43027291,95,YORK MILLS,888282,WEST - 95S YORK MILLS SHORT TURN towards MEADO...,2,26:01:00,1 days 02:01:00,3892,10522,Ellesmere Rd at Muirbank Blvd,43.793969,-79.157635
1193493,43027291,95,YORK MILLS,888282,WEST - 95S YORK MILLS SHORT TURN towards MEADO...,3,26:01:47,1 days 02:01:47,4388,10523,Ellesmere Rd at Meadowvale Rd,43.792623,-79.163292
1193494,43027292,95,YORK MILLS,888282,WEST - 95S YORK MILLS SHORT TURN towards MEADO...,1,26:14:35,1 days 02:14:35,10122,10521,Cul De Sac (Loop) at Ellesmere Rd,43.794687,-79.154799
1193495,43027292,95,YORK MILLS,888282,WEST - 95S YORK MILLS SHORT TURN towards MEADO...,2,26:15:00,1 days 02:15:00,3892,10522,Ellesmere Rd at Muirbank Blvd,43.793969,-79.157635
1193496,43027292,95,YORK MILLS,888282,WEST - 95S YORK MILLS SHORT TURN towards MEADO...,3,26:15:47,1 days 02:15:47,4388,10523,Ellesmere Rd at Meadowvale Rd,43.792623,-79.163292


In [287]:
schedule[schedule.shape_id == 888274]

,trip_id,route_short_name,route_long_name,shape_id,trip_headsign,stop_sequence,stop_time,stop_time_delta,stop_id,stop_code,stop_name,stop_lat,stop_lon
1182639,43027101,95,YORK MILLS,888274,WEST - 95 YORK MILLS towards YORK MILLS STATION,1,5:41:13,0 days 05:41:13,10122,10521,Cul De Sac (Loop) at Ellesmere Rd,43.794687,-79.154799
1182640,43027101,95,YORK MILLS,888274,WEST - 95 YORK MILLS towards YORK MILLS STATION,2,5:41:48,0 days 05:41:48,3892,10522,Ellesmere Rd at Muirbank Blvd,43.793969,-79.157635
1182641,43027101,95,YORK MILLS,888274,WEST - 95 YORK MILLS towards YORK MILLS STATION,3,5:42:55,0 days 05:42:55,4388,10523,Ellesmere Rd at Meadowvale Rd,43.792623,-79.163292
1182642,43027101,95,YORK MILLS,888274,WEST - 95 YORK MILLS towards YORK MILLS STATION,4,5:43:43,0 days 05:43:43,7788,10524,Ellesmere Rd at Zaph Ave,43.791840,-79.166565
1182643,43027101,95,YORK MILLS,888274,WEST - 95 YORK MILLS towards YORK MILLS STATION,5,5:44:30,0 days 05:44:30,4807,10525,Ellesmere Rd at Scarboro Ave,43.791060,-79.169811
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1193486,43027290,95,YORK MILLS,888274,WEST - 95 YORK MILLS towards YORK MILLS STATION,66,25:44:45,1 days 01:44:45,1852,9074,York Mills Rd at Old Yonge St,43.744926,-79.399283
1193487,43027290,95,YORK MILLS,888274,WEST - 95 YORK MILLS towards YORK MILLS STATION,67,25:45:00,1 days 01:45:00,9572,3265,York Mills Rd at York Ridge Rd,43.744470,-79.401416
1193488,43027290,95,YORK MILLS,888274,WEST - 95 YORK MILLS towards YORK MILLS STATION,68,25:45:25,1 days 01:45:25,6062,11334,York Mills Rd at Old York Mills Rd,43.744461,-79.404964
1193489,43027290,95,YORK MILLS,888274,WEST - 95 YORK MILLS towards YORK MILLS STATION,69,25:45:31,1 days 01:45:31,919,9906,York Mills Rd at Yonge St,43.744417,-79.405968


In [288]:
stop_times = pd.read_feather('data/store/stop_times.ftr')

In [290]:
trips = pd.read_feather('data/store/trips.ftr')

In [302]:
trips[trips.shape_id == 888274]

,route_id,service_id,trip_id,trip_headsign,direction_id,block_id,shape_id,bikes_allowed
36784,65004,1,43027105,WEST - 95 YORK MILLS towards YORK MILLS STATION,1,1860166,888274,1
36785,65004,1,43027104,WEST - 95 YORK MILLS towards YORK MILLS STATION,1,1860167,888274,1
36786,65004,1,43027106,WEST - 95 YORK MILLS towards YORK MILLS STATION,1,1860178,888274,1
36788,65004,1,43027103,WEST - 95 YORK MILLS towards YORK MILLS STATION,1,1860169,888274,1
36789,65004,1,43027102,WEST - 95 YORK MILLS towards YORK MILLS STATION,1,1860168,888274,1
...,...,...,...,...,...,...,...,...
129190,65004,4,43114729,WEST - 95 YORK MILLS towards YORK MILLS STATION,1,1863747,888274,1
129191,65004,4,43114728,WEST - 95 YORK MILLS towards YORK MILLS STATION,1,1863787,888274,1
129192,65004,4,43114727,WEST - 95 YORK MILLS towards YORK MILLS STATION,1,1863746,888274,1
129193,65004,4,43114726,WEST - 95 YORK MILLS towards YORK MILLS STATION,1,1863719,888274,1


In [303]:
trips[trips.shape_id == 888282]

,route_id,service_id,trip_id,trip_headsign,direction_id,block_id,shape_id,bikes_allowed
36975,65004,1,43027291,WEST - 95S YORK MILLS SHORT TURN towards MEADO...,1,1860195,888282,1
36976,65004,1,43027292,WEST - 95S YORK MILLS SHORT TURN towards MEADO...,1,1860192,888282,1
70867,65004,2,43059292,WEST - 95S YORK MILLS SHORT TURN towards MEADO...,1,1861640,888282,1
70868,65004,2,43059293,WEST - 95S YORK MILLS SHORT TURN towards MEADO...,1,1861653,888282,1
99616,65004,3,43086550,WEST - 95S YORK MILLS SHORT TURN towards MEADO...,1,1862760,888282,1
129195,65004,4,43114758,WEST - 95S YORK MILLS SHORT TURN towards MEADO...,1,1863744,888282,1


In [308]:
trips[trips.shape_id == 888272]

,route_id,service_id,trip_id,trip_headsign,direction_id,block_id,shape_id,bikes_allowed
36601,65004,1,43026963,EAST - 95A YORK MILLS towards PORT UNION,0,1860181,888272,1
36606,65004,1,43026962,EAST - 95A YORK MILLS towards PORT UNION,0,1860165,888272,1
36608,65004,1,43026980,EAST - 95A YORK MILLS towards PORT UNION,0,1860182,888272,1
36610,65004,1,43026984,EAST - 95A YORK MILLS towards PORT UNION,0,1860175,888272,1
36611,65004,1,43026985,EAST - 95A YORK MILLS towards PORT UNION,0,1860176,888272,1
...,...,...,...,...,...,...,...,...
129066,65004,4,43114573,EAST - 95A YORK MILLS towards PORT UNION,0,1863746,888272,1
129067,65004,4,43114572,EAST - 95A YORK MILLS towards PORT UNION,0,1863719,888272,1
129068,65004,4,43114571,EAST - 95A YORK MILLS towards PORT UNION,0,1863720,888272,1
129069,65004,4,43114570,EAST - 95A YORK MILLS towards PORT UNION,0,1863745,888272,1


In [304]:
trips[trips.block_id == 1860195]

,route_id,service_id,trip_id,trip_headsign,direction_id,block_id,shape_id,bikes_allowed
36710,65004,1,43027039,EAST - 95C YORK MILLS towards ELLESMERE STATION,0,1860195,888270,1
36725,65004,1,43027059,EAST - 95A YORK MILLS towards PORT UNION,0,1860195,888272,1
36747,65004,1,43027081,EAST - 95B YORK MILLS towards U OF T SCARBOROUGH,0,1860195,888271,1
36760,65004,1,43027086,EAST - 95A YORK MILLS towards PORT UNION,0,1860195,888272,1
36770,65004,1,43027053,EAST - 95A YORK MILLS towards PORT UNION,0,1860195,888272,1
36780,65004,1,43027043,EAST - 95A YORK MILLS towards PORT UNION,0,1860195,888272,1
36889,65004,1,43027151,WEST - 95 YORK MILLS towards YORK MILLS STATION,1,1860195,888274,1
36910,65004,1,43027251,WEST - 95C YORK MILLS towards YORK MILLS STATION,1,1860195,888280,1
36928,65004,1,43027197,WEST - 95 YORK MILLS towards YORK MILLS STATION,1,1860195,888274,1
36943,65004,1,43027210,WEST - 95 YORK MILLS towards YORK MILLS STATION,1,1860195,888281,1


In [307]:
schedule[schedule.trip_id == 43027203]

,trip_id,route_short_name,route_long_name,shape_id,trip_headsign,stop_sequence,stop_time,stop_time_delta,stop_id,stop_code,stop_name,stop_lat,stop_lon
1188499,43027203,95,YORK MILLS,888274,WEST - 95 YORK MILLS towards YORK MILLS STATION,1,22:44:40,0 days 22:44:40,10122,10521,Cul De Sac (Loop) at Ellesmere Rd,43.794687,-79.154799
1188500,43027203,95,YORK MILLS,888274,WEST - 95 YORK MILLS towards YORK MILLS STATION,2,22:45:06,0 days 22:45:06,3892,10522,Ellesmere Rd at Muirbank Blvd,43.793969,-79.157635
1188501,43027203,95,YORK MILLS,888274,WEST - 95 YORK MILLS towards YORK MILLS STATION,3,22:45:56,0 days 22:45:56,4388,10523,Ellesmere Rd at Meadowvale Rd,43.792623,-79.163292
1188502,43027203,95,YORK MILLS,888274,WEST - 95 YORK MILLS towards YORK MILLS STATION,4,22:46:28,0 days 22:46:28,7788,10524,Ellesmere Rd at Zaph Ave,43.791840,-79.166565
1188503,43027203,95,YORK MILLS,888274,WEST - 95 YORK MILLS towards YORK MILLS STATION,5,22:46:59,0 days 22:46:59,4807,10525,Ellesmere Rd at Scarboro Ave,43.791060,-79.169811
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1188564,43027203,95,YORK MILLS,888274,WEST - 95 YORK MILLS towards YORK MILLS STATION,66,23:24:45,0 days 23:24:45,1852,9074,York Mills Rd at Old Yonge St,43.744926,-79.399283
1188565,43027203,95,YORK MILLS,888274,WEST - 95 YORK MILLS towards YORK MILLS STATION,67,23:25:00,0 days 23:25:00,9572,3265,York Mills Rd at York Ridge Rd,43.744470,-79.401416
1188566,43027203,95,YORK MILLS,888274,WEST - 95 YORK MILLS towards YORK MILLS STATION,68,23:25:25,0 days 23:25:25,6062,11334,York Mills Rd at Old York Mills Rd,43.744461,-79.404964
1188567,43027203,95,YORK MILLS,888274,WEST - 95 YORK MILLS towards YORK MILLS STATION,69,23:25:31,0 days 23:25:31,919,9906,York Mills Rd at Yonge St,43.744417,-79.405968


In [305]:
schedule[schedule.trip_id == 43027275]

,trip_id,route_short_name,route_long_name,shape_id,trip_headsign,stop_sequence,stop_time,stop_time_delta,stop_id,stop_code,stop_name,stop_lat,stop_lon
1192371,43027275,95,YORK MILLS,888274,WEST - 95 YORK MILLS towards YORK MILLS STATION,1,24:24:40,1 days 00:24:40,10122,10521,Cul De Sac (Loop) at Ellesmere Rd,43.794687,-79.154799
1192372,43027275,95,YORK MILLS,888274,WEST - 95 YORK MILLS towards YORK MILLS STATION,2,24:25:06,1 days 00:25:06,3892,10522,Ellesmere Rd at Muirbank Blvd,43.793969,-79.157635
1192373,43027275,95,YORK MILLS,888274,WEST - 95 YORK MILLS towards YORK MILLS STATION,3,24:25:56,1 days 00:25:56,4388,10523,Ellesmere Rd at Meadowvale Rd,43.792623,-79.163292
1192374,43027275,95,YORK MILLS,888274,WEST - 95 YORK MILLS towards YORK MILLS STATION,4,24:26:28,1 days 00:26:28,7788,10524,Ellesmere Rd at Zaph Ave,43.791840,-79.166565
1192375,43027275,95,YORK MILLS,888274,WEST - 95 YORK MILLS towards YORK MILLS STATION,5,24:26:59,1 days 00:26:59,4807,10525,Ellesmere Rd at Scarboro Ave,43.791060,-79.169811
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1192436,43027275,95,YORK MILLS,888274,WEST - 95 YORK MILLS towards YORK MILLS STATION,66,25:04:45,1 days 01:04:45,1852,9074,York Mills Rd at Old Yonge St,43.744926,-79.399283
1192437,43027275,95,YORK MILLS,888274,WEST - 95 YORK MILLS towards YORK MILLS STATION,67,25:05:00,1 days 01:05:00,9572,3265,York Mills Rd at York Ridge Rd,43.744470,-79.401416
1192438,43027275,95,YORK MILLS,888274,WEST - 95 YORK MILLS towards YORK MILLS STATION,68,25:05:25,1 days 01:05:25,6062,11334,York Mills Rd at Old York Mills Rd,43.744461,-79.404964
1192439,43027275,95,YORK MILLS,888274,WEST - 95 YORK MILLS towards YORK MILLS STATION,69,25:05:31,1 days 01:05:31,919,9906,York Mills Rd at Yonge St,43.744417,-79.405968


In [306]:
schedule[schedule.trip_id == 43027291]

,trip_id,route_short_name,route_long_name,shape_id,trip_headsign,stop_sequence,stop_time,stop_time_delta,stop_id,stop_code,stop_name,stop_lat,stop_lon
1193491,43027291,95,YORK MILLS,888282,WEST - 95S YORK MILLS SHORT TURN towards MEADO...,1,26:00:35,1 days 02:00:35,10122,10521,Cul De Sac (Loop) at Ellesmere Rd,43.794687,-79.154799
1193492,43027291,95,YORK MILLS,888282,WEST - 95S YORK MILLS SHORT TURN towards MEADO...,2,26:01:00,1 days 02:01:00,3892,10522,Ellesmere Rd at Muirbank Blvd,43.793969,-79.157635
1193493,43027291,95,YORK MILLS,888282,WEST - 95S YORK MILLS SHORT TURN towards MEADO...,3,26:01:47,1 days 02:01:47,4388,10523,Ellesmere Rd at Meadowvale Rd,43.792623,-79.163292


In [309]:
h

,shape_id,stop_sequence,next_stop_duration,next_stop_seconds,route_short_name,route_long_name,trip_headsign,stop_id,stop_code,stop_name,stop_lat,stop_lon,next_stop_id
1182411,888273,1,0 days 00:02:46,166.0,95,YORK MILLS,EAST - 95A YORK MILLS towards PORT UNION,10351,10519,Ellesmere Rd at Meadowvale Rd,43.792312,-79.163843,9463.0
1182420,888273,2,0 days 00:01:49,109.0,95,YORK MILLS,EAST - 95A YORK MILLS towards PORT UNION,9463,10520,Ellesmere Rd at Muirbank Blvd,43.793732,-79.157941,10122.0
1182429,888273,3,0 days 00:03:46,226.0,95,YORK MILLS,EAST - 95A YORK MILLS towards PORT UNION,10122,10521,Cul De Sac (Loop) at Ellesmere Rd,43.794687,-79.154799,24410.0
1182438,888273,4,0 days 00:06:08,368.0,95,YORK MILLS,EAST - 95A YORK MILLS towards PORT UNION,24410,16023,Kingston Rd at Rylander Blvd,43.796887,-79.149251,1002.0
1182447,888273,5,NaT,NaN,95,YORK MILLS,EAST - 95A YORK MILLS towards PORT UNION,1002,7198,Sheppard Ave East at Kingston Rd North Side,43.799459,-79.146775,10122.0


In [311]:
g.tail(5)

,shape_id,stop_sequence,next_stop_duration,next_stop_seconds,route_short_name,route_long_name,trip_headsign,stop_id,stop_code,stop_name,stop_lat,stop_lon,next_stop_id
1181911,888272,64,0 days 00:02:27.500000,147.5,95,YORK MILLS,EAST - 95A YORK MILLS towards PORT UNION,10351,10519,Ellesmere Rd at Meadowvale Rd,43.792312,-79.163843,9463.0
1182011,888272,65,0 days 00:01:49,109.0,95,YORK MILLS,EAST - 95A YORK MILLS towards PORT UNION,9463,10520,Ellesmere Rd at Muirbank Blvd,43.793732,-79.157941,10122.0
1182111,888272,66,0 days 00:03:46,226.0,95,YORK MILLS,EAST - 95A YORK MILLS towards PORT UNION,10122,10521,Cul De Sac (Loop) at Ellesmere Rd,43.794687,-79.154799,24410.0
1182211,888272,67,0 days 00:06:08,368.0,95,YORK MILLS,EAST - 95A YORK MILLS towards PORT UNION,24410,16023,Kingston Rd at Rylander Blvd,43.796887,-79.149251,1002.0
1182311,888272,68,NaT,NaN,95,YORK MILLS,EAST - 95A YORK MILLS towards PORT UNION,1002,7198,Sheppard Ave East at Kingston Rd North Side,43.799459,-79.146775,10351.0


In [312]:
d

,shape_id,stop_sequence,next_stop_duration,next_stop_seconds,route_short_name,route_long_name,trip_headsign,stop_id,stop_code,stop_name,stop_lat,stop_lon,next_stop_id
1193491,888282,1,0 days 00:00:25,25.0,95,YORK MILLS,WEST - 95S YORK MILLS SHORT TURN towards MEADO...,10122,10521,Cul De Sac (Loop) at Ellesmere Rd,43.794687,-79.154799,3892.0
1193493,888282,2,0 days 00:00:47,47.0,95,YORK MILLS,WEST - 95S YORK MILLS SHORT TURN towards MEADO...,3892,10522,Ellesmere Rd at Muirbank Blvd,43.793969,-79.157635,4388.0
1193495,888282,3,NaT,NaN,95,YORK MILLS,WEST - 95S YORK MILLS SHORT TURN towards MEADO...,4388,10523,Ellesmere Rd at Meadowvale Rd,43.792623,-79.163292,3169.0


In [313]:
a.head(3)

,shape_id,stop_sequence,next_stop_duration,next_stop_seconds,route_short_name,route_long_name,trip_headsign,stop_id,stop_code,stop_name,stop_lat,stop_lon,next_stop_id
1182456,888274,1,0 days 00:00:30.500000,30.50,95,YORK MILLS,WEST - 95 YORK MILLS towards YORK MILLS STATION,10122,10521,Cul De Sac (Loop) at Ellesmere Rd,43.794687,-79.154799,3892.0
1182563,888274,2,0 days 00:00:58.500000,58.50,95,YORK MILLS,WEST - 95 YORK MILLS towards YORK MILLS STATION,3892,10522,Ellesmere Rd at Muirbank Blvd,43.793969,-79.157635,4388.0
1182670,888274,3,0 days 00:00:41.750000,41.75,95,YORK MILLS,WEST - 95 YORK MILLS towards YORK MILLS STATION,4388,10523,Ellesmere Rd at Meadowvale Rd,43.792623,-79.163292,7788.0


In [314]:
seq[seq.stop_name == 'Finch Station']

,shape_id,stop_sequence,next_stop_duration,next_stop_seconds,route_short_name,route_long_name,trip_headsign,stop_id,stop_code,stop_name,stop_lat,stop_lon,next_stop_id
152431,886593,15,NaT,NaN,125,DREWRY,EAST - 125 DREWRY towards FINCH STATION,14212,14692,Finch Station,43.781430,-79.415193,2137.0
152528,886595,1,0 days 00:00:27.500000,27.5,125,DREWRY,WEST - 125 DREWRY towards BATHURST (TORRESDALE),14212,14692,Finch Station,43.781430,-79.415193,1805.0
465628,887173,59,NaT,NaN,36,FINCH WEST,EAST - 36 FINCH WEST towards FINCH STATION,14206,14686,Finch Station,43.780290,-79.414537,8023.0
472920,887180,30,NaT,NaN,36,FINCH WEST,EAST - 36 FINCH WEST towards FINCH STATION,14206,14686,Finch Station,43.780290,-79.414537,3979.0
472956,887181,32,NaT,NaN,36,FINCH WEST,EAST - 36 FINCH WEST towards FINCH STATION,14206,14686,Finch Station,43.780290,-79.414537,4534.0
473015,887183,59,NaT,NaN,36,FINCH WEST,EAST - 36 FINCH WEST towards FINCH STATION,14206,14686,Finch Station,43.780290,-79.414537,8023.0
476943,887188,22,NaT,NaN,36,FINCH WEST,EAST - 36 FINCH WEST towards FINCH STATION,14206,14686,Finch Station,43.780290,-79.414537,14206.0
477122,887190,1,0 days 00:01:52,112.0,36,FINCH WEST,WEST - 36S FINCH WEST SHORT TURN towards ARROW RD,14206,14686,Finch Station,43.780290,-79.414537,7753.0
485465,887196,1,0 days 00:01:06,66.0,36,FINCH WEST,WEST - 36B FINCH WEST towards HUMBERWOOD,14206,14686,Finch Station,43.780290,-79.414537,7753.0
487077,887202,1,0 days 00:01:18,78.0,36,FINCH WEST,WEST - 36B FINCH WEST towards HUMBERWOOD,14206,14686,Finch Station,43.780290,-79.414537,7753.0
